In [1]:
from data.loader import fetch_spot_history, fetch_option_quotes, clean_option_quotes

# 1. Spot history
df_spot = fetch_spot_history("SPY", start="2021-01-01", end="2021-01-10")
display(df_spot.head(), df_spot.shape)

# 2. Option quotes
df_opts = fetch_option_quotes("SPY")
display(df_opts.head(), df_opts.shape)

# 3. Cleaned quotes
df_clean = clean_option_quotes(df_opts)
display(df_clean.head(), df_clean.shape)


YF.download() has changed argument auto_adjust default to True


Price,date,Open,High,Low,Close,Volume
0,2021-01-04,353.394019,353.525858,343.516585,347.254761,110210800
1,2021-01-05,346.605116,350.748176,346.558018,349.646484,66426200
2,2021-01-06,348.121144,354.966638,347.565600,351.736908,107997700
3,2021-01-07,354.137981,357.716072,353.959074,356.962799,68766800
4,2021-01-08,358.365694,359.213133,355.079499,358.996582,71677200


(5, 6)

,strike,bid,ask,mid_price,mid_iv,expiry,type
0,473.0,90.34,92.47,91.405,0.941407,2025-05-12,call
1,474.0,89.34,91.47,90.405,0.929688,2025-05-12,call
2,481.0,82.34,84.47,83.405,0.859376,2025-05-12,call
3,492.0,71.34,73.47,72.405,0.750003,2025-05-12,call
4,500.0,63.35,65.47,64.410,0.677738,2025-05-12,call


(7213, 7)

,strike,bid,ask,mid_price,mid_iv,expiry,type
0,473.0,90.34,92.47,91.405,0.941407,2025-05-12,call
1,474.0,89.34,91.47,90.405,0.929688,2025-05-12,call
2,481.0,82.34,84.47,83.405,0.859376,2025-05-12,call
3,492.0,71.34,73.47,72.405,0.750003,2025-05-12,call
4,500.0,63.35,65.47,64.410,0.677738,2025-05-12,call


(7099, 7)